In [27]:
#Send Ether to a Contract
import time
from web3 import Web3, HTTPProvider

contract_address     = '0x0ea14c47be22fed7e7220876998be2249bf5a9e3'
wallet_private_key   = 'e3557bf7cf16ef32d5fb9040971b6342038e71a71c06b0f477af049e7bf21d72'
wallet_address       = '0x042D6E3AC0018098d331093Ed6C4aE4C143CC568'

w3 = Web3(HTTPProvider('https://ropsten.infura.io/v3/16fd0fe69c1441b4ba6abc7f50891fcc'))

w3.eth.enable_unaudited_features()
contract_address = w3.toChecksumAddress(contract_address)
wallet_address = w3.toChecksumAddress(wallet_address)

def send_ether_to_contract(amount_in_ether):

    amount_in_wei = w3.toWei(amount_in_ether,'ether');

    nonce = w3.eth.getTransactionCount(wallet_address)
    print(nonce)
    print(amount_in_wei)

    txn_dict = {
            'to': contract_address,
            'value': amount_in_wei,
            'gas': 2000000,
            'gasPrice': w3.toWei('40', 'gwei'),
            'nonce': nonce,
            'chainId': 3
    }

    signed_txn = w3.eth.account.signTransaction(txn_dict, wallet_private_key)

    txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)

    txn_receipt = None
    count = 0
    while txn_receipt is None and (count < 30):

        txn_receipt = w3.eth.getTransactionReceipt(txn_hash)

        print(txn_receipt)

        time.sleep(10)


    if txn_receipt is None:
        return {'status': 'failed', 'error': 'timeout'}

    return {'status': 'added', 'txn_receipt': txn_receipt}

if __name__ == "__main__":
    send_ether_to_contract(0.03)

c:\users\hchen\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: enable_unaudited_features is deprecated in favor of doing nothing at all
  # Remove the CWD from sys.path while we load stuff.


2
30000000000000000
None
None
AttributeDict({'blockHash': HexBytes('0xefc0ab49490f1940e4ce5148cb031c1cf136be351f978f674a6642a929cf7feb'), 'blockNumber': 5480108, 'contractAddress': None, 'cumulativeGasUsed': 41421, 'from': '0x042d6e3ac0018098d331093ed6c4ae4c143cc568', 'gasUsed': 41421, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 1, 'to': '0x0ea14c47be22fed7e7220876998be2249bf5a9e3', 'transactionHash': HexBytes('0xaa7aa3b017ac68cf6c178a14f96985171f2a9a40e71015bc449fd7913e5ab836

In [32]:
#Calling a function in a smart contract
import contract_abi
import time
from web3 import Web3, HTTPProvider

contract_address     = '0x0ea14c47be22fed7e7220876998be2249bf5a9e3'
wallet_private_key   = 'e3557bf7cf16ef32d5fb9040971b6342038e71a71c06b0f477af049e7bf21d72'
wallet_address       = '0x042D6E3AC0018098d331093Ed6C4aE4C143CC568'

w3 = Web3(HTTPProvider('https://ropsten.infura.io/v3/16fd0fe69c1441b4ba6abc7f50891fcc'))

w3.eth.enable_unaudited_features()
contract_address = w3.toChecksumAddress(contract_address)
wallet_address = w3.toChecksumAddress(wallet_address)

contract = w3.eth.contract(address = contract_address, abi = contract_abi.abi)

def check_whether_address_is_approved(address):
    return contract.functions.isApproved(address).call()

if __name__ == "__main__":
    is_approved = check_whether_address_is_approved(wallet_address)
    print(is_approved)


c:\users\hchen\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: enable_unaudited_features is deprecated in favor of doing nothing at all
  if sys.path[0] == '':


True


In [33]:
#Transacting with a function
import contract_abi
import time
from web3 import Web3, HTTPProvider

contract_address     = '0x0ea14c47be22fed7e7220876998be2249bf5a9e3'
wallet_private_key   = 'e3557bf7cf16ef32d5fb9040971b6342038e71a71c06b0f477af049e7bf21d72'
wallet_address       = '0x042D6E3AC0018098d331093Ed6C4aE4C143CC568'

w3 = Web3(HTTPProvider('https://ropsten.infura.io/v3/16fd0fe69c1441b4ba6abc7f50891fcc'))

w3.eth.enable_unaudited_features()
contract_address = w3.toChecksumAddress(contract_address)
wallet_address = w3.toChecksumAddress(wallet_address)

contract = w3.eth.contract(address = contract_address, abi = contract_abi.abi)


def broadcast_an_opinion(covfefe):


    nonce = w3.eth.getTransactionCount(wallet_address)

    txn_dict = contract.functions.broadcastOpinion(covfefe).buildTransaction({
        'chainId': 3,
        'gas': 140000,
        'gasPrice': w3.toWei('40', 'gwei'),
        'nonce': nonce,
    })

    signed_txn = w3.eth.account.signTransaction(txn_dict, private_key=wallet_private_key)

    result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)

    tx_receipt = w3.eth.getTransactionReceipt(result)

    count = 0
    while tx_receipt is None and (count < 30):

        time.sleep(10)

        tx_receipt = w3.eth.getTransactionReceipt(result)

        print(tx_receipt)


    if tx_receipt is None:
        return {'status': 'failed', 'error': 'timeout'}

    processed_receipt = contract.events.OpinionBroadcast().processReceipt(tx_receipt)

    print(processed_receipt)
    
    output = "Address {} broadcasted the opinion: {}"\
        .format(processed_receipt[0].args._soapboxer, processed_receipt[0].args._opinion)
    print(output)

    return {'status': 'added', 'processed_receipt': processed_receipt}

if __name__ == "__main__":

    broadcast_an_opinion('Will you still love me')


c:\users\hchen\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: enable_unaudited_features is deprecated in favor of doing nothing at all
  if sys.path[0] == '':


AttributeDict({'blockHash': HexBytes('0x7f67d490d312ec05c06fa8b9acefc2204834f1035226229d930fade8c53a1b77'), 'blockNumber': 5480534, 'contractAddress': None, 'cumulativeGasUsed': 93004, 'from': '0x042d6e3ac0018098d331093ed6c4ae4c143cc568', 'gasUsed': 47263, 'logs': [AttributeDict({'address': '0x0EA14C47Be22fEd7e7220876998BE2249Bf5a9e3', 'blockHash': HexBytes('0x7f67d490d312ec05c06fa8b9acefc2204834f1035226229d930fade8c53a1b77'), 'blockNumber': 5480534, 'data': '0x000000000000000000000000042d6e3ac0018098d331093ed6c4ae4c143cc5680000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000001657696c6c20796f75207374696c6c206c6f7665206d6500000000000000000000', 'logIndex': 1, 'removed': False, 'topics': [HexBytes('0xcda4350c176dee701be26e34bb6ddef641e5f6847b5ff6ca83ccca7faa85ddaf')], 'transactionHash': HexBytes('0x549ab258b8301c887e1125a4b4671ec0242766a0f96bcf9d439aa2c34ee3b1c9'), 'transactionIndex': 1})], 'logsBloom': HexBytes('0x